In [46]:
import pandas as pd
import numpy as np
import re

In [47]:
# The path to our CSV files
csv_path_2011 = "../data/Medicare_Provider_Charge_Inpatient_DRG100_FY2011.csv"
csv_path_2012 = "../data/Medicare_Provider_Charge_Inpatient_DRG100_FY2012.csv"
csv_path_2013 = "../data/Medicare_Provider_Charge_Inpatient_DRG100_FY2013.csv"
csv_path_2014 = "../data/Medicare_Provider_Charge_Inpatient_DRGALL_FY2014.csv"
csv_path_2015 = "../data/Medicare_Provider_Charge_Inpatient_DRGALL_FY2015.csv"

# Read our data into pandas
df_2011 = pd.read_csv(csv_path_2011)
df_2011['year'] = '2011'
df_2012 = pd.read_csv(csv_path_2012)
df_2012['year'] = '2012'
df_2013 = pd.read_csv(csv_path_2013)
df_2013['year'] = '2013'
df_2014 = pd.read_csv(csv_path_2014)
df_2014['year'] = '2014'
df_2015 = pd.read_csv(csv_path_2015)
df_2015['year'] = '2015'

df = pd.concat([df_2011,df_2012,df_2013,df_2014,df_2015])

In [48]:
drg_titles = set(df['DRG Definition'].unique())
print(len(drg_titles),drg_titles)

598 {'156 - OTHER EAR, NOSE, MOUTH & THROAT DIAGNOSES W/O CC/MCC', '758 - INFECTIONS, FEMALE REPRODUCTIVE SYSTEM W CC', '607 - MINOR SKIN DISORDERS W/O MCC', '699 - OTHER KIDNEY & URINARY TRACT DIAGNOSES W CC', '580 - OTHER SKIN, SUBCUT TISS & BREAST PROC W CC', '138 - MOUTH PROCEDURES W/O CC/MCC', '915 - ALLERGIC REACTIONS W MCC', '880 - ACUTE ADJUSTMENT REACTION & PSYCHOSOCIAL DYSFUNCTION', '175 - PULMONARY EMBOLISM W MCC', '456 - SPINAL FUS EXC CERV W SPINAL CURV/MALIG/INFEC OR 9+ FUS W MCC', '184 - MAJOR CHEST TRAUMA W CC', '060 - MULTIPLE SCLEROSIS & CEREBELLAR ATAXIA W/O CC/MCC', '623 - SKIN GRAFTS & WOUND DEBRID FOR ENDOC, NUTRIT & METAB DIS W CC', '133 - OTHER EAR, NOSE, MOUTH & THROAT O.R. PROCEDURES W CC/MCC', '200 - PNEUMOTHORAX W CC', '189 - PULMONARY EDEMA & RESPIRATORY FAILURE', '760 - MENSTRUAL & OTHER FEMALE REPRODUCTIVE SYSTEM DISORDERS W CC/MCC', '168 - OTHER RESP SYSTEM O.R. PROCEDURES W/O CC/MCC', '479 - BIOPSIES OF MUSCULOSKELETAL SYSTEM & CONNECTIVE TISSUE W/O CC/

In [49]:
patterns= [' W/O CC/MCC',' W/O MCC',' W MCC', ' W CC', ' MCC', '/MCC']
df['mod_DRG Definition'] = df['DRG Definition'].replace(patterns, "", regex=True)

mod_drg_titles = set(df['mod_DRG Definition'].unique())
print(len(mod_drg_titles),mod_drg_titles)

598 {'696 - KIDNEY & URINARY TRACT SIGNS & SYMPTOMS', '250 - PERC CARDIOVASC PROC W/O CORONARY ARTERY STENT', '040 - PERIPH/CRANIAL NERVE & OTHER NERV SYST PROC', '102 - HEADACHES', '557 - TENDONITIS, MYOSITIS & BURSITIS', '880 - ACUTE ADJUSTMENT REACTION & PSYCHOSOCIAL DYSFUNCTION', '856 - POSTOPERATIVE OR POST-TRAUMATIC INFECTIONS W O.R. PROC', '829 - MYELOPROLIF DISORD OR POORLY DIFF NEOPL W OTHER O.R. PROC', '255 - UPPER LIMB & TOE AMPUTATION FOR CIRC SYSTEM DISORDERS', '687 - KIDNEY & URINARY TRACT NEOPLASMS', '256 - UPPER LIMB & TOE AMPUTATION FOR CIRC SYSTEM DISORDERS', '314 - OTHER CIRCULATORY SYSTEM DIAGNOSES', '073 - CRANIAL & PERIPHERAL NERVE DISORDERS', '260 - CARDIAC PACEMAKER REVISION EXCEPT DEVICE REPLACEMENT', '189 - PULMONARY EDEMA & RESPIRATORY FAILURE', '949 - AFTERCARE', '194 - SIMPLE PNEUMONIA & PLEURISY', '717 - OTHER MALE REPRODUCTIVE SYSTEM O.R. PROC EXC MALIGNANCY', '976 - HIV W MAJOR RELATED CONDITION', '244 - PERMANENT CARDIAC PACEMAKER IMPLANT', '951 - OTHER

In [50]:
mod_drgs = df.groupby('mod_DRG Definition')
len(mod_drgs.groups.keys()),mod_drgs.groups.keys()

(598,
 dict_keys(['001 - HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SYSTEM', '002 - HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SYSTEM', '003 - ECMO OR TRACH W MV 96+ HRS OR PDX EXC FACE, MOUTH & NECK W MAJ O.R.', '003 - ECMO OR TRACH W MV >96 HRS OR PDX EXC FACE, MOUTH & NECK W MAJ O.R.', '004 - TRACH W MV 96+ HRS OR PDX EXC FACE, MOUTH & NECK W/O MAJ O.R.', '005 - LIVER TRANSPLANT OR INTESTINAL TRANSPLANT', '006 - LIVER TRANSPLANT', '007 - LUNG TRANSPLANT', '008 - SIMULTANEOUS PANCREAS/KIDNEY TRANSPLANT', '011 - TRACHEOSTOMY FOR FACE,MOUTH & NECK DIAGNOSES', '012 - TRACHEOSTOMY FOR FACE,MOUTH & NECK DIAGNOSES', '013 - TRACHEOSTOMY FOR FACE,MOUTH & NECK DIAGNOSES', '014 - ALLOGENEIC BONE MARROW TRANSPLANT', '016 - AUTOLOGOUS BONE MARROW TRANSPLANT', '017 - AUTOLOGOUS BONE MARROW TRANSPLANT', '020 - INTRACRANIAL VASCULAR PROCEDURES W PDX HEMORRHAGE', '023 - CRANIO W MAJOR DEV IMPL/ACUTE COMPLEX CNS PDX OR CHEMO IMPLANT', '024 - CRANIO W MAJOR DEV IMPL/ACUTE COMPLEX CNS PDX', '025 - CR